In [12]:
#import dependencies

import pandas as pd
from bs4 import BeautifulSoup as bs
#from bs4.element import Comment
#from splinter import Browser
import requests
import json
from config import api_key
#import time
#import numpy as np


## Extract Data

In [13]:
#datasource url 
url = "https://inciweb.nwcg.gov/feeds/rss/incidents/"
state_fetch_url = "https://maps.googleapis.com/maps/api/geocode/json?latlng="
api_key = "&key=" + api_key

In [14]:
def state_extract(obj):
    stateval = ""
    
    for eachitem in obj:
        for k, v in eachitem.items():
            if k == "long_name":
                stateval = v
            elif isinstance(v, list):
                for item in v:
                    if item == "administrative_area_level_1":
                        return stateval

    return stateval

In [18]:
# instanciate landing dataframe
starting_data = pd.DataFrame()

# request xml code from url
xml_data = requests.get(url).content

#parse the xml response
soup = bs(xml_data, "xml")
     
# Find all text in the data
texts = str(soup.findAll(text=True)).replace('\\n','')
    
#Find the tag/child
child = soup.find("item")

# instatiate column lists
title = []
published = []
lat = []
lon = []
link = []
description = []
state=[]

#loop trough each "item" in the xml response and store the target data
while True:    
    try:
        title.append(" ".join(child.find('title')))
    except:
         title.append(" ")
            
    try:
        published.append(" ".join(child.find('published')))
    except:
        published.append(" ")
        
    try:
        lat.append(" ".join(child.find('geo:lat')))
    except:
        lat.append(" ")
            
    try:
        lon.append(" ".join(child.find('geo:long')))
    except:
        lon.append(" ")
        
    try:
        link.append(" ".join(child.find('link')))
    except:
        link.append(" ")
        
    try:
        description.append(" ".join(child.find('description')))
    except:
        description.append(" ")
    
    try:
        latlng = " ".join(child.find('geo:lat')) + ","+ " ".join(child.find('geo:long'))
        resp_data = requests.get(state_fetch_url + latlng + api_key).json()
        state.append(state_extract(resp_data.get('results')[0].get('address_components')))
    except:
        state.append(" ")
    
    try:   
        # Next sibling of child, here: 'item' 
        child = child.find_next_sibling('item')
    except:
        break
    
    #create dataframe
    data = pd.DataFrame({"title":title,
                                    "published":published,
                                    "lat":lat,
                                    "lon":lon,
                                    "link_url":link,
                                    "description": description,
                                    "state": state})
    starting_data = starting_data.append(data, ignore_index = True)

In [19]:
# drop duplicate rows
unique_data = starting_data.drop_duplicates(keep="first",ignore_index="True")
unique_data.head(15)

,title,published,lat,lon,link_url,description,state
0,Bond Fire (Wildfire),"Mon, 07 Dec 2020 21:01:28 -06:00",33.777222222222,-117.63833333333,http://inciweb.nwcg.gov/incident/7275/,"The Bond Fire started December 2, 2020 at 10:1...",California
1,Canyon Wildfire (Wildfire),"Mon, 07 Dec 2020 20:44:22 -06:00",35.881944444444,-118.45805555556,http://inciweb.nwcg.gov/incident/7276/,,California
2,North Complex (Wildfire),"Mon, 07 Dec 2020 17:50:15 -06:00",40.090833333333,-120.93111111111,http://inciweb.nwcg.gov/incident/6997/,"Incident Start Date: 8/17/2020 Size: 318,93...",California
3,Fall Prescribed Fire Projects 2020 (Prescribed...,"Mon, 07 Dec 2020 15:50:50 -06:00",44.417777777778,-118.95138888889,http://inciweb.nwcg.gov/incident/7216/,Please take a moment to look at the links sec...,Oregon
4,Klamath NF RX Burning 2020/2021 (Prescribed Fire),"Mon, 07 Dec 2020 15:26:48 -06:00",41.738611111111,-122.77888888889,http://inciweb.nwcg.gov/incident/7274/,"After a challenging fire season, the arrival o...",California
5,Cameron Peak Fire (Wildfire),"Mon, 07 Dec 2020 10:48:34 -06:00",40.608611111111,-105.87916666667,http://inciweb.nwcg.gov/incident/6964/,A Type 3 Incident Management Team assumed comm...,Colorado
6,Rattlesnake Fire (Wildfire),"Sun, 06 Dec 2020 19:33:15 -06:00",36.419722222222,-118.44833333333,http://inciweb.nwcg.gov/incident/7131/,The Rattlesnake Fire was discovered on August ...,California
7,SQF Complex (Wildfire),"Sun, 06 Dec 2020 11:58:25 -06:00",36.255,-118.49666666667,http://inciweb.nwcg.gov/incident/7048/,Operations Map | Fire History Map | Land Ow...,California
8,Grizzly Creek Fire (Wildfire),"Fri, 04 Dec 2020 09:05:15 -06:00",39.566666666667,-107.27138888889,http://inciweb.nwcg.gov/incident/6942/,Dec. 4 Update. There has been little activity ...,Colorado
9,Creek Fire (Wildfire),"Thu, 03 Dec 2020 18:06:37 -06:00",37.201111111111,-119.27166666667,http://inciweb.nwcg.gov/incident/7147/,,California


In [20]:
# go the the link url for each rown and extract additional data (cause, size)

#instatiate landing lists
causes = []
sizes = []

#loop through each row of data
for x in range(len(unique_data)):
    #find the link in the row
    url = starting_data.loc[x,"link_url"]
    #go to the page and grap all the tables
    tables = pd.read_html(url)
    
    #the number of tables the page has will determine which tables we look in for data.
    #if there are more than one tables...
    if len(tables)>1:
        try:
            # find the "cause" in the first table on the page (if it exists)
            cause = tables[0].iloc[2,1]
        except:
            cause = "unknown"
    
        try:
            # find the "size" in the second table on the page (if it exists)
            size = tables[1].loc[(tables[1][0]) == "Size",1].item()            
        except:
            size = "unknown"
    #if there is only one table on the page...
    else:
        try:
            # find the "cause" in the first table on the page (if it exists)
            cause = tables[0].iloc[1,1]
            # no size data is available
            size = "n/a"
        except:
            cause = "unknown"
    
    #add cause and size to their lists
    causes.append(cause)
    sizes.append(size)
    
    #print progress
    print(f"{x+1} of {len(unique_data)}")


1 of 126
2 of 126
3 of 126
4 of 126
5 of 126
6 of 126
7 of 126
8 of 126
9 of 126
10 of 126
11 of 126
12 of 126
13 of 126
14 of 126
15 of 126
16 of 126
17 of 126
18 of 126
19 of 126
20 of 126
21 of 126
22 of 126
23 of 126
24 of 126
25 of 126
26 of 126
27 of 126
28 of 126
29 of 126
30 of 126
31 of 126
32 of 126
33 of 126
34 of 126
35 of 126
36 of 126
37 of 126
38 of 126
39 of 126
40 of 126
41 of 126
42 of 126
43 of 126
44 of 126
45 of 126
46 of 126
47 of 126
48 of 126
49 of 126
50 of 126
51 of 126
52 of 126
53 of 126
54 of 126
55 of 126
56 of 126
57 of 126
58 of 126
59 of 126
60 of 126
61 of 126
62 of 126
63 of 126
64 of 126
65 of 126
66 of 126
67 of 126
68 of 126
69 of 126
70 of 126
71 of 126
72 of 126
73 of 126
74 of 126
75 of 126
76 of 126
77 of 126
78 of 126
79 of 126
80 of 126
81 of 126
82 of 126
83 of 126
84 of 126
85 of 126
86 of 126
87 of 126
88 of 126
89 of 126
90 of 126
91 of 126
92 of 126
93 of 126
94 of 126
95 of 126
96 of 126
97 of 126
98 of 126
99 of 126
100 of 126
101 of 1

## Clean/Transform Data 

In [21]:
# if cause has the word "investigation" in it, set cause to "unknown"
for y in range(len(causes)):
    if "Investigation" in causes[y]:
        causes[y] = "Unknown"

# remove the word "Acres" from the size data
sizes = [s.replace(" Acres","") for s in sizes]
sizes = [s.replace(",","") for s in sizes]

In [22]:
# add causes and sizes to the dataframe
unique_data["cause"] = causes
unique_data["acres"] = sizes

<ipython-input-22-9acd8c5b95cc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_data["cause"] = causes
<ipython-input-22-9acd8c5b95cc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_data["acres"] = sizes


In [23]:
# see the counts of each fire cause for reference
grouped_df = unique_data.groupby(["cause"])
grouped_df.count()

,title,published,lat,lon,link_url,description,state,acres
cause,,,,,,,,
Burned Area Emergency Response,1,1,1,1,1,1,1,1
Human,12,12,12,12,12,12,12,12
Lightning,31,31,31,31,31,31,31,31
Prescribed Fire,30,30,30,30,30,30,30,30
Unknown,52,52,52,52,52,52,52,52


In [24]:
state_grouped_df = unique_data.groupby(["state"])
state_grouped_df.count()

,title,published,lat,lon,link_url,description,cause,acres
state,,,,,,,,
Arizona,17,17,17,17,17,17,17,17
California,37,37,37,37,37,37,37,37
Colorado,15,15,15,15,15,15,15,15
Idaho,13,13,13,13,13,13,13,13
Montana,9,9,9,9,9,9,9,9
Nevada,2,2,2,2,2,2,2,2
New Mexico,4,4,4,4,4,4,4,4
Oregon,15,15,15,15,15,15,15,15
Utah,4,4,4,4,4,4,4,4


In [25]:
# save the dataframe as "clean_data"

clean_data = unique_data
clean_data.head()

,title,published,lat,lon,link_url,description,state,cause,acres
0,Bond Fire (Wildfire),"Mon, 07 Dec 2020 21:01:28 -06:00",33.777222222222,-117.63833333333,http://inciweb.nwcg.gov/incident/7275/,"The Bond Fire started December 2, 2020 at 10:1...",California,Unknown,7375
1,Canyon Wildfire (Wildfire),"Mon, 07 Dec 2020 20:44:22 -06:00",35.881944444444,-118.45805555556,http://inciweb.nwcg.gov/incident/7276/,,California,Unknown,7375
2,North Complex (Wildfire),"Mon, 07 Dec 2020 17:50:15 -06:00",40.090833333333,-120.93111111111,http://inciweb.nwcg.gov/incident/6997/,"Incident Start Date: 8/17/2020 Size: 318,93...",California,Unknown,83
3,Fall Prescribed Fire Projects 2020 (Prescribed...,"Mon, 07 Dec 2020 15:50:50 -06:00",44.417777777778,-118.95138888889,http://inciweb.nwcg.gov/incident/7216/,Please take a moment to look at the links sec...,Oregon,Unknown,7375
4,Klamath NF RX Burning 2020/2021 (Prescribed Fire),"Mon, 07 Dec 2020 15:26:48 -06:00",41.738611111111,-122.77888888889,http://inciweb.nwcg.gov/incident/7274/,"After a challenging fire season, the arrival o...",California,Unknown,83


In [27]:
#store as csv for testing
clean_data.to_csv("data.csv")